In [ ]:
# default_exp process

# Multithreading

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import mmcv
def multi_thread(fn, array_inputs, max_workers=None, desc="Multi-thread Pipeline", unit="Samples", verbose=True):
    from concurrent.futures import ThreadPoolExecutor
    from functools import partial
    from glob import glob
    from multiprocessing import Pool
    from tqdm import tqdm

    def _wraper(x):
        i, input = x
        return {i: fn(input)}

    array_inputs = [(i, _) for i, _ in enumerate(array_inputs)]
    if verbose:
        progress_bar = mmcv.utils.ProgressBar(len(array_inputs))
    outputs = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for result in executor.map(_wraper, array_inputs):
            outputs.update(result)
            if verbose:
                progress_bar.update()
    if verbose:
        print('Finished')
    outputs = list(outputs.values())
    return outputs


## Example

In [ ]:
# from avcv.process import multi_thread
show_doc(multi_thread)


# from glob import glob
# import mmcv
# import numpy as np
# from avcv.process import multi_thread
# from tqdm import tqdm


# def f(x):
#     mmcv.imread(x, channel_order='rgb')
#     return None

# paths = glob('../sample-imgs/*')
# inputs = np.random.choice(paths, 100)
# fast_imgs = multi_thread(f, inputs)

<h4 id="multi_thread" class="doc_header"><code>multi_thread</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>multi_thread</code>(**`fn`**, **`array_inputs`**, **`max_workers`**=*`None`*, **`desc`**=*`'Multi-thread Pipeline'`*, **`unit`**=*`'Samples'`*, **`verbose`**=*`True`*)



In [ ]:
from tqdm import tqdm
import time
import concurrent

def dosomething(t):
    time.sleep(t)
    return f'Sleep for {t} second'

start = time.time()

inputs = [1,4,2,5]
# pbar = tqdm(len(inputs))
with concurrent.futures.ProcessPoolExecutor(max_workers=3) as executor:
    future_results = []
    for inp in inputs:
        future_results.append(executor.submit(dosomething, inp))
        
    results = []
    for i in range(6):
        time.sleep(1)
        results = [f.result() for f in future_results if f.done()]
        print(i, results)
#     for result in tqdm(future_results):
#         print(result.done())
#         results.append(result.result())
#         print(results[-1], time.time()-start)
    
finish = time.time()
print("Total: ", finish-start)

0 []
1 ['Sleep for 1 second']
2 ['Sleep for 1 second', 'Sleep for 2 second']
3 ['Sleep for 1 second', 'Sleep for 2 second']
4 ['Sleep for 1 second', 'Sleep for 4 second', 'Sleep for 2 second']
5 ['Sleep for 1 second', 'Sleep for 4 second', 'Sleep for 2 second', 'Sleep for 5 second']
Total:  6.031975269317627


In [ ]:
import torch.multiprocessing as mp
import torch
import torch.nn as nn
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.param = nn.Parameter(torch.Tensor([0]), requires_grad=False)

In [ ]:
model = MyModel()

In [ ]:
model.param

Parameter containing:
tensor([0.])

In [ ]:
train(model)

In [ ]:


def train(model):
    model.param += 1

num_processes = 4
model = MyModel()
model.share_memory()
processes = []
for rank in range(num_processes):
    p = mp.Process(target=train, args=(model,))
    p.start()
    processes.append(p)
    
for p in processes:
    p.join()

In [ ]:
model.param

Parameter containing:
tensor([4.])